In [1]:
import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import re
from unidecode import unidecode

In [2]:
def make_soup(url):
#     r = requests.get(url)
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    r = urlopen(req).read()
    soup = BeautifulSoup(r, "html")
    return soup

# put urls in a list
def get_xml_urls(soup):
    urls = [loc.string for loc in soup.find_all("loc")]
    return urls

# get media urls
def get_urls(xml):
    list_of_urls = []
    soup = make_soup(xml)
    urls = get_xml_urls(soup)
    # loop through the urls
    for url in urls:
        list_of_urls.append(url)
    return list_of_urls

In [3]:
imdb_pro = "https://www.imdb.com/title/tt2338151/"

In [5]:
content_list = []
content_dict = {}
soup = make_soup(imdb_pro)

try:
    name = soup.find("h1","").text
    content_dict["NAME"] = (unidecode(name).split(" ")[0])   
except:
    content_dict["NAME"] = ""
try:
    content_dict["RATING"] = soup.find("span",itemprop="ratingValue").text + "/10"
except:
    content_dict["RATING"] = ""
try:
    content_dict["VOTER COUNT"] = soup.find("span",itemprop="ratingCount").text
except:
    content_dict["VOTER COUNT"] = ""
try:
    content_dict["DURATION"] = soup.find("time",datetime="PT153M").text.strip()
except:
    content_dict["DURATION"] = ""
try:
    content_dict["RELEASE DATE"] = soup.find("a",title="See more release dates").text.split("(")[0].strip()
except:
    content_dict["RELEASE DATE"] = ""
try:
    content_dict["RELEASE LOCATION"] = soup.find("a",title="See more release dates").text.split("(")[1].replace(")\n","")
except:
    content_dict["RELEASE LOCATION"] = ""
try:
    content_dict["POSTER LINK"] = soup.find("div","poster").find("img")["src"]
except:
    content_dict["POSTER LINK"] = ""
try:
    content_dict["TRAILER LINK"] = "https://www.imdb.com" + soup.find("div","videoPreview__videoContainer").find("a")["href"]
except:
    content_dict["TRAINER LINK"] = ""
try:
    content_dict["OVERVIEW"] = soup.find("div" ,"summary_text").text.strip()
except:
    content_dict["OVERVIEW"] = ""

try:
    dir = soup.find("div", "plot_summary")
    dir2 = dir.findAll("div", "credit_summary_item")
    s = dir2[0].text.split(":")
    content_dict['DIRECTOR'] = s[1].strip("\n")
    
    s = dir2[1].text.split(":")
    content_dict['WRITER'] = s[1].strip("\n")
   
    s = dir2[2].text.split(":")
    content_dict['STARS'] = s[1].strip("\n")
except:
    content_dict['DIRECTOR'] = ''
    content_dict['WRITER'] = ''
    content_dict['STARS'] = ''
    
try:
    x = soup.findAll("tr","odd")
    y = soup.findAll("tr","even")
    
    castlist = []   
    
    for item in x:
        try:
            castdict = {}
            actor = item.find("td","primary_photo").find('img')
            castdict['ACTOR'] = actor['alt']
            role = item.find("td","character").text.strip()
            castdict['ROLE'] = role
            castlist.append(castdict)
        except:
            continue
      
    for item in y:
        try:
            castdict = {}
            actor = item.find("td","primary_photo").find('img')
            castdict['ACTOR'] = actor['alt']
            role = item.find("td","character").text.strip()
            castdict['ROLE'] = role
            castlist.append(castdict)
        except:
            continue
            
    content_dict['CAST'] = castlist
    
except:
    content_dict['CAST'] = ''

try:
    more=soup.find("div", attrs={"class":"rec_slide"})
    mov=[]
    mov=more.findAll("a",href=re.compile("title"))
    name=[]
    for i in range(len(mov)):
        ttl=mov[i].find("img")
        name.append(ttl["alt"])
    content_dict['MORE LIKE THIS'] = name
except:
    content_dict['MORE LIKE THIS'] = ''
    
try:
    story=soup.find("div",attrs={"id":"titleStoryLine"})
    line=story.find("div",attrs={"class","inline canwrap"}).p.span
    storyline=line.text
    content_dict['STORYLINE'] = storyline
except:
    content_dict['STORYLINE'] = ''

    
content_list.append(content_dict)

In [6]:
movies_df = pd.DataFrame.from_dict(content_list, orient='columns')
movies_df

,NAME,RATING,VOTER COUNT,DURATION,RELEASE DATE,RELEASE LOCATION,POSTER LINK,TRAILER LINK,OVERVIEW,DIRECTOR,WRITER,STARS,CAST,MORE LIKE THIS,STORYLINE
0,PK,8.1/10,"156,165",2h 33min,19 December 2014,India,https://m.media-amazon.com/images/M/MV5BMTYzOT...,https://www.imdb.com/video/imdb/vi2912071193?p...,An alien on Earth loses the only device he can...,Rajkumar Hirani,"Rajkumar Hirani, Abhijat Joshi","Aamir Khan, Anushka Sharma, Sanjay Dutt |\nSee...","[{'ACTOR': 'Aamir Khan', 'ROLE': 'PK'}, {'ACTO...","[3 Idiots, Dangal, Lagaan: Once Upon a Time in...",P. K. is a comedy of ideas about a strange...


In [7]:
movies_df.to_csv("IMDB_movies.csv")